# `ipytest` Summary

[ipytest](https://github.com/chmp/ipytest/tree/main) allows you to run [pytest](https://pytest.org) in Jupyter notebooks. `ipytest` aims to give access to the full `pytest` experience to make it easy to transfer tests out of notebooks into separate test files.

To get started install `ipytest` via

In [ ]:
%pip install -U ipytest

To use `ipytest`, import it and configure the notebook:

In [2]:
import ipytest

ipytest.autoconfig()

In most cases, running `ipytest.autoconfig()` will result in reasonable defaults:

- register the `%%ipytest` magic to execute tests
- register the `pytest` assert rewriter with the notebook to get nice assert messages 

For more control, pass the relevant arguments to `ipytest.autconfig()`. For details, see the reference in the readme.

## Execute tests

To execute test, just decorate the cells containing tests with the `%%ipytest` magic:

In [2]:
%%ipytest

# define the tests

def test_my_func():
    assert my_func(0) == 0
    assert my_func(1) == 0
    assert my_func(2) == 2
    assert my_func(3) == 2
    
    
def my_func(x):
    return x // 2 * 2 

.                                                                                            [100%]
1 passed in 0.02s


To execute tests without IPython magics use the `ipytest.run` function

In [3]:
ipytest.run()


no tests ran in 0.01s


<ExitCode.NO_TESTS_COLLECTED: 5>

## Using pytest fixtures

Common pytest features, such as fixtures and parametrize, are supported out of the box:

In [4]:
%%ipytest

import pytest

@pytest.mark.parametrize('input,expected', [
    (0, 0),
    (1, 0),
    (2, 2),
    (3, 2),
])
def test_parametrized(input, expected):
    assert my_func(input) == expected
    
    
@pytest.fixture
def my_fixture():
    return 42
    
    
def test_fixture(my_fixture):
    assert my_fixture == 42

FFFF.                                                                                        [100%]
============================================= FAILURES =============================================
______________________________________ test_parametrized[0-0] ______________________________________

input = 0, expected = 0

    @pytest.mark.parametrize('input,expected', [
        (0, 0),
        (1, 0),
        (2, 2),
        (3, 2),
    ])
    def test_parametrized(input, expected):
>       assert my_func(input) == expected
E       NameError: name 'my_func' is not defined

/var/folders/1_/swvbhq057ls2d5l5wv0zs_qr0000gn/T/ipykernel_36565/41091767.py:10: NameError
______________________________________ test_parametrized[1-0] ______________________________________

input = 1, expected = 0

    @pytest.mark.parametrize('input,expected', [
        (0, 0),
        (1, 0),
        (2, 2),
        (3, 2),
    ])
    def test_parametrized(input, expected):
>       assert my_func(input) == e

## Selecting tests

[pytest](https://pytest.org) offers a [extensive options](https://docs.pytest.org/en/latest/how-to/usage.html#specifying-which-tests-to-run) to select subsets of tests to run. They can be also used in conjunction with `ipytest`:

In [5]:
%%ipytest -k feature1

def test_feature1_test1():
    assert True

def test_feature1_test2():
    assert True
    
def test_feature2_test1():
    assert False
    
def test_feature2_test2():
    assert False

..                                                                                           [100%]
2 passed, 2 deselected in 0.01s


Tests can also be selected based on node ids. The notebook can be referenced via the special `{MODULE}` name. In addition, it is possible to generate node ids for tests inside the notebook via the `{test_name}` shorthand. For example `{test_feature1_test1}` references the corresponding test defined in the notebook: 

In [6]:
%%ipytest {test_feature1_test1}
        
def test_feature1_test1():
    assert True

def test_feature1_test2():
    assert False

.                                                                                            [100%]
1 passed in 0.03s


[Deselection](https://docs.pytest.org/en/7.1.x/example/pythoncollection.html#deselect-tests-during-test-collection) works as well:

In [1]:
%%ipytest --deselect {test_feature1_test2}

def test_feature1_test1():
    assert True

def test_feature1_test2():
    assert False

UsageError: Cell magic `%%ipytest` not found.


## Debugging failed tests

The [debugging functionality of pytest](https://docs.pytest.org/en/latest/how-to/failures.html) can be used as well. For example, to debug the first failed test (and then stop the pytest run) use:

In [ ]:
%%ipytest -x --pdb

def test_example():
    for i in range(10):
        if i == 5: 
            raise ValueError(i)

## Checking notebooks automatically

`ipytest` itself does not support validating notebooks in a programmatic fashion. For this task, the [`nbval` package](https://nbval.readthedocs.io/en/latest) can be used. In its default configuration, `ipytest` will not raise lead to execution failures, but only display the exception. While this behavior is helpful during interactive development, it will prevent `nbval` from catching errors. To raise errors if any test fails, configure `raise_on_error=True`:

In [ ]:
%%ipytest
# ipytest: raise_on_error=True

def test():
    raise ValueError()